# I'm Blocked

In performing my [Gold in the Badlands](https://github.com/OpenBagTwo/AngerLocalBeehive/blob/f849268a0854b7216b846bc08f6538cd8dc2f047/notebooks/Gold%20in%20the%20Badlands.ipynb) analysis, the block counts were just not making sense. So I've created a sparkling new superflat world to see if maybe it's an issue with just that world or if something is legitimately broken in the `nbt` library.

In this superflat world, I have placed two blocks:
1. A block of ancient debris at (1, 2, 3)
2. A block of gilded blackstone at (4, 5, 6)

## Imports, Setup and Macros

In [1]:
import json
from collections import Counter, defaultdict
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict, List, Tuple

import matplotlib.pyplot as plt
import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
save_folder = Path(environ["SAVE_PATH"])

In [3]:
def summarize_keystore(keystore: Dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: Dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, Dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [4]:
def split_block_value(value: int) -> List[int]:
    """Split a 64-bit int into 16x 4-bit ints

    Parameters
    ----------
    value: 64-bit int
        The int to split

    Returns
    -------
    list of 16x 4-bit ints
    """
    small_ints: List[int] = []
    for _ in range(16):
        value, remainder = divmod(value, 2**4)
        small_ints.insert(0, remainder)
    return small_ints

In [9]:
def split_into_coords(idx: int) -> tuple[int, int, int]:
    """Convert an index into three coordinates.
    We don't yet know which coordinate is which.

    Parameters
    ----------
    idx : int
        A number between 0 and 4095 (inclusive)

    Returns
    -------
    (int, int, int) tuple
        The three coordinate values separated.
    """
    coord_1, remainder = divmod(idx, 16 * 16)
    coord_2, remainder = divmod(remainder, 16)
    return (coord_1, coord_2, remainder)

In [5]:
region_data = region.RegionFile(save_folder / "region" / "r.0.0.mca")
chunk = region_data.get_chunk(0, 0)
chunklet = chunk["sections"][4]
assert chunklet["Y"].value == 0

In [6]:
palette = [block["Name"].value for block in chunklet["block_states"]["palette"]]
blocks: list[int] = sum(
    [split_block_value(v) for v in chunklet["block_states"]["data"]],
    [],
)
block_count = {}
for idx, count in sorted(Counter(blocks).items()):
    block_count[palette[idx]] = count
display(block_count)

{'minecraft:deepslate': 256,
 'minecraft:stone': 1278,
 'minecraft:ancient_debris': 1,
 'minecraft:gilded_blackstone': 1,
 'minecraft:dirt': 1280,
 'minecraft:gravel': 1280}

Okay, well here it's working--it must have just been an issue with that world. Let's now try to get the coordinates of these placed blocks.

In [11]:
debris_idx = blocks.index(palette.index("minecraft:ancient_debris"))
gilded_idx = blocks.index(palette.index("minecraft:gilded_blackstone"))
print(debris_idx, gilded_idx)

574 1387


Now let's split those out into the three 4-bit values that should give us the coordinates.

In [13]:
debris_coords = split_into_coords(debris_idx)
gilded_coords = split_into_coords(gilded_idx)
print(debris_coords, gilded_coords)

(2, 3, 14) (5, 6, 11)


So I think what this is speaking to is that if I reverse the `split_block_value` direction, then the coordinates are in the form (y, z, x)

In [14]:
blocks = sum(
    [list(reversed(split_block_value(v))) for v in chunklet["block_states"]["data"]],
    [],
)
debris_idx = blocks.index(palette.index("minecraft:ancient_debris"))
gilded_idx = blocks.index(palette.index("minecraft:gilded_blackstone"))
debris_coords = split_into_coords(debris_idx)
gilded_coords = split_into_coords(gilded_idx)
print(debris_idx, "->", debris_coords)
print(gilded_idx, "->", gilded_coords)

561 -> (2, 3, 1)
1380 -> (5, 6, 4)


That's it.